In [1]:
import re
import networkx as nx
import matplotlib.pyplot as plt
import random
import matplotlib.cm as cm
import yaml
import matplotlib.colors as mcolors
import pandas as pd
import subprocess
import sys
build_dir = '/Users/letshopethisworks2/CLionProjects/SALBP_ILS/cmake-build-python_interface/'
sys.path.insert(0, build_dir)
build_dir = '/home/jot240/DADA/SALBP_ILS/build/'
sys.path.insert(0, build_dir)
sys.path.append('../src')
sys.path.append('../src/constraint_disc')
from metrics.node_and_edge_features import randomized_kahns_algorithm
from alb_instance_compressor import *
from constraint_disc.lstd import *
from constraint_disc.constraint_discovery import *
from constraint_disc.beam_search import remove_edges
from SALBP_solve import salbp1_bbr_call, parse_alb_results_new_bbr,salbp1_mhh_solve

from copy import deepcopy
import ILS_ALBP

In [2]:
#test_problem = parse_alb("/Users/letshopethisworks2/Documents/phd_paper_material/DADA/small data set_n=20/instance_n=20_16.alb")
albp_dicts = alb_dicts = open_salbp_pickle("/home/jot240/DADA/DADA/data/raw/pkl_datasets/otto/small.pkl" )
test_problem = alb_dicts[5]

In [3]:
test_problem

{'num_tasks': 20,
 'cycle_time': 1000,
 'task_times': {'1': 581,
  '2': 283,
  '3': 555,
  '4': 470,
  '5': 787,
  '6': 473,
  '7': 392,
  '8': 570,
  '9': 633,
  '10': 616,
  '11': 294,
  '12': 559,
  '13': 622,
  '14': 323,
  '15': 439,
  '16': 506,
  '17': 380,
  '18': 352,
  '19': 463,
  '20': 444},
 'precedence_relations': [['1', '3'],
  ['1', '4'],
  ['1', '6'],
  ['2', '3'],
  ['2', '5'],
  ['2', '9'],
  ['2', '10'],
  ['3', '12'],
  ['3', '13'],
  ['4', '9'],
  ['4', '13'],
  ['4', '19'],
  ['5', '8'],
  ['6', '7'],
  ['6', '10'],
  ['7', '11'],
  ['7', '12'],
  ['8', '11'],
  ['9', '20'],
  ['10', '11'],
  ['10', '12'],
  ['10', '13'],
  ['11', '16'],
  ['12', '14'],
  ['12', '15'],
  ['13', '16'],
  ['13', '17'],
  ['14', '19'],
  ['16', '20'],
  ['17', '18']],
 'name': PosixPath('/home/jot240/MALBPW/MMABPW/SALBP_benchmark/small data set_n=20/instance_n=20_410.alb')}

In [4]:
def plot_salbp_graph(G):
    # Compute layout
    #pos = nx.nx_agraph.graphviz_layout(G, prog='dot')
    pos = nx.nx_pydot.graphviz_layout(G, prog = "dot")

    # Extract edge probabilities (default to 0 if missing)
    edge_probs = [G[u][v].get('prob', 0) for u, v in G.edges()]
    
    # Normalize for color mapping
    vmin, vmax = min(edge_probs), max(edge_probs)

    # Create figure and axis explicitly
    fig, ax = plt.subplots(figsize=(8, 6))

    # Draw nodes and labels
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=800, edgecolors='black', ax=ax)
    nx.draw_networkx_labels(G, pos, font_size=10, font_color='black', ax=ax)

    # Draw edges with colormap
    edges = nx.draw_networkx_edges(
        G,
        pos,
        edge_color=edge_probs,
        edge_cmap=plt.cm.viridis,
        edge_vmin=vmin,
        edge_vmax=vmax,
        width=2,
        ax=ax
    )

    # Add colorbar *using the same Axes*
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm.set_array([])  # required for colorbar
    fig.colorbar(sm, ax=ax, label="Edge Probability")

    # Final touches
    ax.set_title("SALBP Graph Colored by Edge Probability")
    ax.axis("off")
    plt.tight_layout()
    plt.show()
#plot_salbp_dict(test_problem)

In [5]:
feasible_sequences = get_feasible_seq(test_problem, 2, 42)
print(feasible_sequences)

G_max_close = seqs_to_dag(feasible_sequences)






[['1', '2', '5', '4', '6', '7', '3', '10', '8', '13', '9', '11', '12', '15', '14', '19', '16', '17', '20', '18'], ['2', '5', '1', '4', '9', '6', '8', '3', '7', '10', '11', '12', '14', '13', '17', '18', '19', '16', '15', '20']]


In [6]:

edge_probs = edge_prob_generation(test_problem, G_max_close, seed=42)
nx.set_edge_attributes(G_max_close, edge_probs, name='prob')
G_max_red = nx.transitive_reduction(G_max_close)
nx.set_edge_attributes(G_max_red, edge_probs, name='prob')
nx.set_edge_attributes(G_max_close, 1, name = 't_cost' )
edge_probs

Seed is  42
edges to perturb,  [('9', '19'), ('11', '14'), ('1', '16'), ('2', '3'), ('2', '13'), ('19', '16'), ('12', '14'), ('10', '20'), ('7', '14'), ('10', '16'), ('8', '16'), ('5', '14'), ('1', '18'), ('1', '17'), ('10', '17'), ('17', '18'), ('2', '10'), ('6', '18'), ('9', '18'), ('7', '18'), ('14', '18'), ('9', '11'), ('5', '13'), ('17', '20'), ('5', '20'), ('2', '4'), ('1', '11'), ('12', '19'), ('8', '18'), ('3', '20'), ('7', '20'), ('12', '18'), ('8', '14')]


{('1', '6'): 0.1,
 ('6', '7'): 0.1,
 ('2', '3'): 0.9,
 ('3', '13'): 0.1,
 ('14', '19'): 0.1,
 ('2', '5'): 0.1,
 ('1', '4'): 0.1,
 ('17', '18'): 0.9,
 ('12', '15'): 0.1,
 ('11', '16'): 0.1,
 ('10', '12'): 0.1,
 ('4', '19'): 0.1,
 ('1', '3'): 0.1,
 ('7', '11'): 0.1,
 ('4', '13'): 0.1,
 ('5', '8'): 0.1,
 ('8', '11'): 0.1,
 ('2', '10'): 0.9,
 ('3', '12'): 0.1,
 ('6', '10'): 0.1,
 ('12', '14'): 0.9,
 ('4', '9'): 0.1,
 ('7', '12'): 0.1,
 ('9', '20'): 0.1,
 ('13', '16'): 0.1,
 ('13', '17'): 0.1,
 ('10', '13'): 0.1,
 ('16', '20'): 0.1,
 ('10', '11'): 0.1,
 ('2', '9'): 0.1,
 ('2', '12'): 0.9,
 ('2', '15'): 0.9,
 ('7', '18'): 0.1,
 ('6', '15'): 0.9,
 ('8', '16'): 0.1,
 ('6', '8'): 0.9,
 ('7', '10'): 0.9,
 ('7', '17'): 0.9,
 ('1', '9'): 0.9,
 ('1', '7'): 0.9,
 ('1', '15'): 0.9,
 ('8', '19'): 0.9,
 ('10', '15'): 0.9,
 ('1', '8'): 0.9,
 ('5', '12'): 0.9,
 ('12', '20'): 0.9,
 ('11', '19'): 0.9,
 ('3', '15'): 0.9,
 ('4', '6'): 0.9,
 ('13', '20'): 0.9,
 ('2', '13'): 0.1,
 ('5', '16'): 0.9,
 ('15', '20

In [7]:
set1 = set([('2', '5'), ('4', '6'), ('2', '19'), ('2', '10'), ('12', '15'), ('1', '12'), ('4', '18'), ('7', '9'), ('10', '18'), ('5', '20'), ('6', '12'), ('1', '5'), ('5', '3'), ('8', '17'), ('10', '15'), ('5', '14'), ('7', '18'), ('6', '19'), ('12', '9'), ('7', '14'), ('6', '9'), ('8', '15'), ('8', '13'), ('1', '17'), ('9', '18'), ('12', '20'), ('2', '18'), ('11', '20'), ('7', '12'), ('5', '15')])
set2 = set([('5', '13'), ('4', '9'), ('7', '15'), ('6', '14'), ('2', '15'), ('2', '13'), ('7', '16'), ('8', '17'), ('3', '19'), ('2', '6'), ('13', '16'), ('16', '20'), ('4', '11'), ('2', '14'), ('13', '20'), ('4', '20'), ('6', '16'), ('8', '18'), ('7', '17'), ('14', '19'), ('14', '13'), ('4', '12'), ('5', '11'), ('5', '15'), ('10', '17'), ('6', '13'), ('11', '16'), ('9', '20'), ('7', '9'), ('7', '20')])
set2

{('10', '17'),
 ('11', '16'),
 ('13', '16'),
 ('13', '20'),
 ('14', '13'),
 ('14', '19'),
 ('16', '20'),
 ('2', '13'),
 ('2', '14'),
 ('2', '15'),
 ('2', '6'),
 ('3', '19'),
 ('4', '11'),
 ('4', '12'),
 ('4', '20'),
 ('4', '9'),
 ('5', '11'),
 ('5', '13'),
 ('5', '15'),
 ('6', '13'),
 ('6', '14'),
 ('6', '16'),
 ('7', '15'),
 ('7', '16'),
 ('7', '17'),
 ('7', '20'),
 ('7', '9'),
 ('8', '17'),
 ('8', '18'),
 ('9', '20')}

[('5', '13'), ('4', '9'), ('7', '15'), ('6', '14'), ('2', '15'), ('2', '13'), ('7', '16'), ('8', '17'), ('3', '19'), ('2', '6'), ('13', '16'), ('16', '20'), ('4', '11'), ('2', '14'), ('13', '20'), ('4', '20'), ('6', '16'), ('8', '18'), ('7', '17'), ('14', '19'), ('14', '13'), ('4', '12'), ('5', '11'), ('5', '15'), ('10', '17'), ('6', '13'), ('11', '16'), ('9', '20'), ('7', '9'), ('7', '20')]

In [ ]:
os.chdir('/home/jot240/DADA/DADA/')
xp_config_fp = "data/xp_config/hoff_simple_graph_priorityml.yaml"
xp_config, ml_config, ml_model = load_and_backup_configs(xp_config_fp, backup_folder=False)

mh = constant_weight
#mh = salbp1_hoff_solve
selector_method = "lstd_ml"
ex_fp = '../BBR-for-SALBP1/SALB/SALB/salb'
beam_config= {'width':3, 'depth':3}
#beam_config= {'width':10, 'depth':3}
mh_kwargs = {'ml_config':ml_config}
do_greedy_run(test_problem, 15, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config,  ml_model=ml_model, ml_config=ml_config)

Loaded experiment config from data/xp_config/hoff_simple_graph_priorityml.yaml
{'hoff': {'alpha_iter': 2, 'alpha_size': 0.005, 'beta_iter': 2, 'beta_size': 0.005, 'reverse': True}, 'priority': {'n_random': 100}, 'ml_param_fp': 'data/ml_models/hyper_parameter/features/edge_no_rw_no_edgeeval.yaml'}
Using ML features and model specified in data/ml_models/hyper_parameter/features/edge_no_rw_no_edgeeval.yaml
Loaded ML config from data/ml_models/hyper_parameter/features/edge_no_rw_no_edgeeval.yaml
{'features': ['parent_weight', 'parent_pos_weight', 'child_weight', 'child_pos_weight', 'neighborhood_min', 'neighborhood_max', 'neighborhood_avg', 'neighborhood_std', 'parent_in_degree', 'parent_out_degree', 'child_in_degree', 'child_out_degree', 'chain_avg', 'chain_min', 'chain_max', 'chain_std', 'priority_min_gap', 'priority_max_gap', 'random_spread', 'random_coefficient_of_variation', 'random_avg_gap', 'random_min_gap', 'random_max_gap', 'ti_size', 'prec_bias', 'stage_difference', 'prec_strengt

In [ ]:
G_max_close.edges(data=True)

In [ ]:
# test_problem = parse_alb("/Users/letshopethisworks2/Documents/phd_paper_material/DADA/small data set_n=20/instance_n=20_16.alb")
# #mh = os_weight
# #mh = salbp1_hoff_solve
# mh=constant_weight
# selector_method = "beam_mh"
# ex_fp = "/Users/letshopethisworks2/Documents/phd_paper_material/BBR-for-SALBP1/SALB/SALB/salb"
# beam_config= {'width':3, 'depth':3}
# #beam_config= {'width':1, 'depth':1}
# do_greedy_run(test_problem, 5, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config,)

In [ ]:
mh = constant_weight
#mh = salbp1_hoff_solve
selector_method = "random"
#ex_fp = "/Users/letshopethisworks2/Documents/phd_paper_material/BBR-for-SALBP1/SALB/SALB/salb"
beam_config= {'width':3, 'depth':3}
#beam_config= {'width':10, 'depth':3}
do_greedy_run(test_problem, 15, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config)

In [ ]:
mh = constant_weight
#mh = salbp1_hoff_solve
selector_method = "beam_mh"
beam_config= {'width':3, 'depth':3}
#beam_config= {'width':10, 'depth':3}
do_greedy_run(test_problem, 15, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config)

In [ ]:
#mh = os_weight
mh = salbp1_prioirity_solve
#mh = salbp1_hoff_solve
selector_method = "beam_ml"
#beam_config= {'width':3, 'depth':3}
beam_config= {'width':10, 'depth':3}

xp_config, ml_config, ml_model = load_and_backup_configs(xp_config_fp, backup_folder=None)

do_greedy_run(test_problem, 20, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config, ml_model = ml_model, ml_config=ml_config)

In [ ]:
G_min = nx.DiGraph()
G_min.add_nodes_from(test_problem["task_times"].keys())
mh = salbp1_hoff_solve
G_max_red = nx.transitive_reduction(G_max_close)
G_max_red.add_edges_from((u, v, G_max_close.edges[u, v]) for u, v in G_max_red.edges)
edges = get_possible_edges(G_max_red, G_min)
calc_phi_mh(test_problem, G_max_close,edges, mh,10, old_value = 14, mode='lstd_mh')

In [ ]:
#test_problem = parse_alb("/Users/letshopethisworks2/Documents/phd_paper_material/DADA/small data set_n=20/instance_n=20_16.alb")
#mh = os_weight
#mh = salbp1_prioirity_solve
mh = salbp1_mhh_solve
selector_method = "lstd_prob"
#beam_config= {'width':3, 'depth':3}
beam_config= {'width':10, 'depth':3}
do_greedy_run(test_problem, 10, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config)

In [ ]:
#test_problem = parse_alb("/Users/letshopethisworks2/Documents/phd_paper_material/DADA/small data set_n=20/instance_n=20_16.alb")
#mh = os_weight
#mh = salbp1_prioirity_solve
mh = salbp1_mhh_solve
selector_method = "lstd_mh"
#beam_config= {'width':3, 'depth':3}
beam_config= {'width':10, 'depth':3}
do_greedy_run(test_problem, 10, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config)

node_labels {'1': '1', '2': '2', '5': '3', '4': '4', '7': '5', '6': '6', '3': '7', '10': '8', '12': '9', '11': '10', '9': '11', '8': '12', '14': '13', '13': '14', '16': '15', '15': '16', '17': '17', '18': '18', '20': '19', '19': '20'}


G_max_red edges [('1', '5'), ('2', '5'), ('2', '7'), ('2', '4'), ('5', '3'), ('5', '10'), ('5', '11'), ('5', '12'), ('4', '6'), ('6', '3'), ('6', '10'), ('6', '11'), ('6', '12'), ('7', '3'), ('7', '10'), ('7', '11'), ('7', '12'), ('3', '14'), ('3', '8'), ('12', '14'), ('12', '9'), ('12', '8'), ('8', '13'), ('14', '13'), ('13', '16'), ('13', '18'), ('13', '17'), ('13', '15'), ('9', '16'), ('9', '18'), ('9', '17'), ('9', '15'), ('10', '16'), ('10', '18'), ('10', '17'), ('10', '15'), ('11', '16'), ('11', '18'), ('11', '17'), ('11', '15'), ('16', '20'), ('17', '19'), ('17', '20'), ('15', '19'), ('18', '19')]



In [ ]:
test_problem = parse_alb("/Users/letshopethisworks2/Documents/phd_paper_material/DADA/small data set_n=20/instance_n=20_16.alb")
#mh = os_weight
#mh = salbp1_prioirity_solve
mh = salbp1_mhh_solve
selector_method = "beam_mh"
beam_config= {'width':3, 'depth':3}
#beam_config= {'width':10, 'depth':3}
do_greedy_run(test_problem, 10, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config)

In [ ]:
test_problem = parse_alb("/Users/letshopethisworks2/Documents/phd_paper_material/DADA/small data set_n=20/instance_n=20_16.alb")
#mh = os_weight
#mh = salbp1_prioirity_solve
mh = salbp1_prioirity_solve
selector_method = "beam_mh"
#ex_fp = "/Users/letshopethisworks2/Documents/phd_paper_material/BBR-for-SALBP1/SALB/SALB/salb"
beam_config= {'width':3, 'depth':3}
#beam_config= {'width':10, 'depth':3}
do_greedy_run(test_problem, 10, G_max_close, ex_fp, mh,selector_method, time_limit=1,seed=42,beam_config= beam_config)

original edges:  [('1', '5'), ('2', '7'), ('2', '4'), ('2', '5'), ('5', '11'), ('5', '10'), ('5', '3'), ('5', '12'), ('4', '6'), ('6', '11'), ('6', '10'), ('6', '3'), ('6', '12'), ('7', '11'), ('7', '10'), ('7', '3'), ('7', '12'), ('3', '8'), ('3', '14'), ('12', '8'), ('12', '9'), ('12', '14'), ('8', '13'), ('14', '13'), ('13', '16'), ('13', '17'), ('13', '15'), ('13', '18'), ('9', '16'), ('9', '17'), ('9', '15'), ('9', '18'), ('10', '16'), ('10', '17'), ('10', '15'), ('10', '18'), ('11', '16'), ('11', '17'), ('11', '15'), ('11', '18'), ('16', '20'), ('17', '20'), ('17', '19'), ('15', '19'), ('18', '19')]


In [ ]:
import networkx as nx


In [ ]:
#test_to_remove = [('10', '16'), ('17', '19'), ('2', '5'), ('7', '11')]
#test_to_remove = [('6','3')]
test_to_remove = []
# test_to_remove = [('2', '4'),
#   ('2', '6'),
#   ('5', '3'),
#   ('5', '12'),
#   ('5', '8'),
#   ('5', '10'),
#   ('1', '10'),
#   ('5', '9'),
#   ('1', '12'),
#   ('1', '9'),
#   ('5', '14'),
#   ('5', '13'),
#   ('7', '3'),
#   ('2', '3'),
#   ('7', '12'),
#   ('7', '9'),
#   ('7', '14'),
#   ('3', '8'),
#   ('3', '14'),
# ]
test_gmax_close = G_max_close.copy()
print("before", test_gmax_close)
test_gmax_close.remove_edges_from(test_to_remove)
print("after", test_gmax_close)

G_max_red = nx.transitive_reduction(test_gmax_close)
test_salbp, new_to_old = set_new_edges(G_max_red, test_problem)
orig_bbr = False
if "BBR-for-SALBP1/" in ex_fp:
    orig_bbr = True
res = salbp1_bbr_call(test_salbp,ex_fp, 1, w_bin_pack=True, time_limit=2, orig_bbr=orig_bbr)
print('n_stations', res['n_stations'])
task_station = {}
for i, station in enumerate(res['task_assignments']):
    task_station[new_to_old[str(i+1)]] = station
sorted_dict = dict(sorted(task_station.items(), key=(lambda x: int(x[0]))))
true_assignments = [val for _,  val in sorted_dict.items()]
true_assignments
def check_feasibility_C(orig_problem, assignments, cycle_time=1000):
    station_loads = [0] * max(assignments)
    for task, station in enumerate(assignments):
        
        station_loads[station-1] += orig_problem['task_times'][str(task+1)]
    print("here are the station loads: ", station_loads)
    if max(station_loads)>cycle_time:
        idx, max_val = max(enumerate(station_loads), key=lambda x: x[1])
        print("Issue! ", idx, max_val)
check_feasibility_C(test_problem, true_assignments)


In [ ]:
import numpy as np

# def calc_phi_mh(orig_salbp, G_max_close, G_min,mh,remaining_budget, old_value = 0, mode='lstd_prob', **mhkwargs):
#     G_max_red = nx.transitive_closure(G_max_close)
#     edges = get_possible_edges(G_max_red, G_min)
#     att = 0
#     for i, edge in enumerate(edges):
#                 #This is to avoid repeat computations (i.e. e1->e2, e2->e1)
#         new_removed = [(edge[0], edge[1])]
#         # Store the current removal sequence
#         # Create list without current item
#         if len(edge) ==3:
#             edge_prob = edge[2]
#             edge_cost = 1
#         elif len(edge) ==4:
#             edge_prob = edge[2]
#             edge_cost = edge[3]
#         else:
#             edge_prob = 1
#             edge_cost = 1
#         if mode =="lstd_prob":
#             reward = 1 * edge_prob/edge_cost
            
#         elif mode == "lstd_mh":
#             G_max_red = remove_edges(G_max_close, new_removed)

#             new_salbp, new_to_old = set_new_edges(G_max_red, orig_salbp)
#             res = mh(new_salbp, **mhkwargs)
#             reward =  max( 0, edge_prob*(old_value - res['n_stations']))/edge_cost
#         att += reward
#     future_budget = max(0,remaining_budget- edge_cost)
#     phi = np.zeros(5)
#     phi[0] = att
#     phi[1] = att * np.sqrt(future_budget)
#     phi[2] = att * future_budget
#     phi[3] = np.sqrt(future_budget)
#     phi[4] = future_budget
#     return phi
   


# def train_lstd(orig_salbp, G_max_close_orig,G_min_orig, max_budget, mh, mode='lstd_prob', m=50, seed = 42, discount_factor=0.95, **mhkwargs):
#     theta = np.zeros(5)
#     A = np.zeros((5,5))
#     b = np.zeros(5)
#     rng = random.Random(seed)

#     print("THIS IS G_MIN", G_min_orig)
#     for _ in range(m):
#         G_min = G_min_orig.copy()
#         remaining_budget = max_budget
#         G_max_close = G_max_close_orig.copy()
#         G_max_red = nx.transitive_reduction(G_max_close)
#         G_max_red.add_edges_from((u, v, G_max_close.edges[u, v]) for u, v in G_max_red.edges)
#         edges = get_possible_edges(G_max_red, G_min)
#         phi_0 = calc_phi_mh(orig_salbp, G_max_close, G_min,mh,remaining_budget, old_value = 0, mode=mode, **mhkwargs)
#         while edges and remaining_budget > 0:
#             #initilization
#             best_edge = []
#             best_reward = 0
#             best_weight = 0
#             best_prob = 0
#             best_time = 1
#             #Getting best (i,j)
#             for edge in edges:
#                     new_removed = [(edge[0], edge[1])]
#                     G_max_close_test = G_max_close.copy()
#                     G_max_close_test.remove_edges_from(new_removed)
#                     if mode == 'lstd_mh':
#                         G_max_red = nx.transitive_closure(G_max_close_test)
#                         new_salbp, new_to_old = set_new_edges(G_max_red, orig_salbp)
#                         res = mh(new_salbp, **mhkwargs)
#                         weight = res['n_stations']
#                     else:
#                         weight = 1
                    
#                     phi = calc_phi_mh(orig_salbp, G_max_close_test, G_min,mh,remaining_budget, old_value = 0, mode=mode, **mhkwargs)
#                     reward = edge[2] * ( weight + np.dot(phi, theta) ) + (1-edge[2]) * np.dot(phi_0, theta)
#                     if reward >= best_reward:
#                         best_edge = edge
#                         best_prob = edge[2]
#                         best_weight = weight
#                         best_reward = reward
#                         if len(edge )==4:
#                             best_time = edge[3]
#             #update b
#             b= b + phi_0 * best_prob *best_weight
#             #State update
#             #random state transition
#             G_max_close, G_max_red, G_min, _= uncertain_query_prec_set(G_max_close, G_max_red, G_min, best_edge, rng)
#             #update budget
#             remaining_budget -= best_time
#             #update available edges
#             edges = get_possible_edges(G_max_red, G_min)
#             #update A and phi_0
#             phi_1 = calc_phi_mh(orig_salbp, G_max_close, G_min,mh,remaining_budget, old_value = 0, mode=mode, **mhkwargs)
#             step = phi_1 - discount_factor * phi_0
#             A = A + np.outer(phi_0, step)
#             phi_0 = phi_1
#         #update theta
#         theta = np.linalg.inv(A) @ b
#     return theta

G_min = nx.DiGraph()
G_min.add_nodes_from(G_max_close.nodes(data=True))  # preserve node attributes
G_min
theta = train_lstd(test_problem, G_max_close, G_min, 10, mh=constant_weight)
